In [1]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain.tools import tool
import pandas as pd
import folium
from itertools import cycle

In [2]:


# Define the PromptTemplate
prompt = PromptTemplate(
    template="""
    You are an AI agent responsible for executing tasks based on the user's query. Below is the conversation so far:

    {messages}

    Based on the user's query, choose the appropriate tool:
    
    - If the query is related to get columns, use `get_columns` to extract column names from a CSV file.
    - If the query is related to visualize, use `visualize_geo_points` to generate a visualization map and save it as an HTML file.
    
    Perform the necessary action directly and inform the user of the results. Always include the file path of the output if a file is generated.
    """,
    input_variables=["messages"]
)

In [3]:
@tool
def get_columns(path: str):
    """
    Read csv files and get the names of the columns.
    
    Parameters:
        path: str - Path to the CSV file.
    
    Returns:
        list: List of the column names.
    """
    
    print(pd.read_csv(path).columns)
    return list(pd.read_csv(path).columns)

In [1]:
@tool
def visualize_geo_points(path:str, category_column:str, lat_column='lat', lon_column='lon', output_file='map.html'):
    """
    Visualize geospatial points on a map with category-based pin colors.
    
    Parameters:
        path (str): Path to CSV file.
        category_column (str): Column name for categories (e.g., 'type').
        lat_column (str): Column name for latitude values.
        lon_column (str): Column name for longitude values.
        output_file (str): Name of the output HTML file.
    
    Returns:
        str: Path to the generated map HTML file.
    """
    df = pd.read_csv(path)
    # Predefined color palette
    predefined_colors = [
        'red', 'blue', 'green', 'purple', 'orange', 'darkred',
        'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue'
    ]

    # Assign colors dynamically to unique categories
    unique_categories = df[category_column].unique()
    color_cycle = cycle(predefined_colors)  # Cycle through colors if categories > colors
    color_map = {category: next(color_cycle) for category in unique_categories}

    # Create a Folium map centered on the mean location
    center_lat = df[lat_column].mean()
    center_lon = df[lon_column].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

    # Add points to the map with dynamic colors
    for _, row in df.iterrows():
        # Generate a dynamic popup text
        popup_content = "<br>".join(
            [f"{col}: {row[col]}" for col in df.columns if col not in [lat_column, lon_column]]
        )

        folium.Marker(
            location=[row[lat_column], row[lon_column]],
            popup=popup_content,
            tooltip=row[category_column],
            icon=folium.Icon(color=color_map.get(row[category_column], 'gray'))  # Default to 'gray'
        ).add_to(m)

    # Save map to file
    m.save(output_file)
    return f"Map saved to {output_file}"


NameError: name 'tool' is not defined

In [ ]:

# Load the LLM model
llm = ChatOllama(model='mistral',temperature=0)

# Define tools
tools = [get_columns, visualize_geo_points]

# Connect the LLM with the prompt
llm_prompt = LLMChain(llm=llm, prompt=prompt)

# Define memory to remember the chat history
memory = MemorySaver()

In [6]:
# Create the react agent
agent = create_react_agent(
    tools=tools,
    model=llm,
    state_modifier=prompt,
    checkpointer=memory
)


In [ ]:
# Configuration
config = {"configurable": {"thread_id": "1"}}
# User input
user_input = "Get columns of /home/pc/Desktop/llma_agent/data/wildLifeInKsa.csv and visualize it."

# Prepare the input messages
input_messages = [HumanMessage(user_input)]

# Run the agent
response = agent.invoke({"messages": input_messages},config=config)

# Print the response
print(response["messages"][-1].content)
